In [ ]:
# Orinigal version: by Wencheng Shao, Maya Fields, Ziwei Wang, and Da Yang in Rossbypalooza 2022
# Updated version: by Bowen Fan (bowen27@uchicago.edu)

from scipy.optimize import fsolve
import math 
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


''' This python notebook includes Virtual temperature correction for the MSE equations (Eq7 and Eq6). It also uses the fc minimum
function instead of defining a specific value. Heat capacity correction has been added.'''

def feedback_func(So=1360, Fo = 0, epsilon0 = 0.5, a0 = 0, mm_dry = 0.028, LW = 'feedback', SW = 'feedback', assumption = 'WTG', p_guess=(250,250,200,200,200,20), output = 'default'):
    # Input parameters
    # So: solar constant in W/m2
    # Fo: oceanic heat transport in W/m2
    # mm_dry: molar mass of dry air
    # assumption: weak temperature gradient (WTG) or weak buoyancy gradient (WBG)
    # p_guess: the starting estimate of the solutions
    
    g = 13.7                # gravity in m/s2
    Pc = 6e4                # depth of convection in Pa, called as Pa in the paper 
    Po = 1.01e5             # surface pressure in Pa *(needs to be fixed for steam atmos)
    R_star = 8.3145         # universal gas constant
    k3 = 0.08               # relates the strength of convection to the cloud fraction
    sigma = 5.67e-8 # Stefan-Boltzmann Constant in W/m2/K4 
    Tc = 230                    # cloud emission temperature (FAT)

    def alpha(fc):
        alpha_p = 0.09 + fc - 0.09*fc
        return alpha_p
    
    def cld_frc(Fc):
        if Fc + 1.0 <= 0:
#         print("Error in cloud fraction calculation, Fc+1 = {}, fc set as 0".format(Fc+1.0))
            fc = 0
        else:
            fc = k3*np.log(Fc+1.0)     # expression of cloud fraction versus convection
            if fc >= 1:  # unrealistic case when we get cloud fraction larger than 1 (Fc larger than 2.8e6 W/m2)
                print("Cloud fraction too high, corrected as 1")
                fc = 1
        return fc

    def Tv(T, mass_ratio, w):               # virtual temperature
        Tv = T*((1 + (w/mass_ratio))/(1+w)) # w is the water vapor mixing ratio
        return Tv

    def a(T):
        if SW == 'feedback': # temperature varying absorption
            T_ref = 280   # Temperature of full absorption limit 
            a_ref = 0.9      # Max water vapor absorption to sunlight
            ka = 1e4      # exponetional factor (like CC relation)
            if T <= T_ref:
                a = a_ref*np.exp(-ka*(1/T - 1/T_ref))
            else:
                a = a_ref
        elif SW == 'constant':
            a = a0
        return a
        
    def equations(p):  # Define the equations to solve
        T1, T2, T3, T4, Fa, Fc = p 

        ### Constants
        #print('solving or calculating ...')
        es0 = 611.2         # reference pressure for CC relation
        M = 0.018           # molar mass of water vapor
        To = 273.15         # reference temperature for CC relation
        mass_ratio = M/mm_dry    # mass ratio between vapor and dry air

        H = ((R_star/mm_dry)*T2)/g  # scale height of dry atmosphere       

        Z_a = np.log(Po/Pc)*H       # convection height* (needs to be fixed for steam atmos)
        Cpd = 1005.7                # heat capacity of dry air in J/kg/K
        Cpv = 1996                  # heat capacity of vapor in J/kg/K 
        L = 2.501e6                 # latent heat of vaporization in J/kg

        RH1 = 0.9 # Dayside surface relative humidity
        RH2 = 0.8 # Dayside atmosphere relative humidity
        RH3 = 0.3 # Nightside atmosphere relative humidity

        # water vapor calculations for dayside surface
        es_1 = es0*pow(np.e, L/(R_star/M)*(1/To - 1/T1)) # saturation vapor pressure at dayside surface
        # *needs to be fixed: mass mixing ratio of water assuming saturation (vapor/dry)
        ws_1 = mass_ratio*es_1/(Po - es_1)               
        w_1 = RH1 * ws_1                                 # vapor pressure at dayside surface
        q1 = w_1/(1+w_1)                                 # specific humidity at dayside surface
        q1_s = ws_1/(1+ws_1)                             # saturation specific humidity at dayside surf (vapor/total)

        # water vapor calcaulations for dayside atmosphere
        es_2 = es0*pow(np.e, L/(R_star/M)*(1/To - 1/T2))
        ws_2 = mass_ratio*es_2/(Pc - es_2) #saturated mixing ratio
        w_2 = RH2 * ws_2
        q2 = w_2/(1+w_2)
        q2_s = ws_2/(1+ws_2)

        # water vapor calculations for nightside atmosphere
        es_3 = es0*pow(np.e, L/(R_star/M)*(1/To - 1/T3))
        ws_3 = mass_ratio*es_3/(Pc - es_3) #saturated mixing ratio
        w_3 = RH3 * ws_3
        q3 = w_3/(1+w_3)
        Cp1 = Cpd*(1 - q1) + Cpv * q1  # specific heat of mixed air at dayside surf
        Cp2 = Cpd*(1 - q2) + Cpv * q2  # specific heat of mixed air at dayside atmos
        
        if LW == 'feedback':
            k2 = 5000 # Relates water vapor to the infrared opacity (need to be checked)

            epsilon_2 = 1 - np.exp(-k2*q2) # clear sky water vapor emissivity (need to be fixed)
            epsilon_3 = 1 - np.exp(-k2*q3)
        elif LW == 'constant':
            epsilon_2 = epsilon0
            epsilon_3 = epsilon0

        k1 = 0.2  # Fraction of heat transport to the nightside deposited in the boundary layer
        Fd = k1*Fa   # a fraction of atmos heat transport in the boundary layer

        # Corrected dayside surface energy equation
        Eq2 = (1/2)*So*(1-alpha(cld_frc(Fc)))*(1-a(T2)) - Fc - Fo + (1-cld_frc(Fc))*epsilon_2*sigma*T2**4 + cld_frc(Fc)*sigma*Tc**4 - sigma*T1**4

        # Corrected dayside atmos energy equation
        Eq3 = (1/2)*So*(1-alpha(cld_frc(Fc)))*a(T2) + Fc - Fa + (1-cld_frc(Fc))*epsilon_2*sigma*T1**4 + cld_frc(Fc)*sigma*T1**4 - 2*(1-cld_frc(Fc))*epsilon_2*sigma*T2**4 - 2*cld_frc(Fc)*sigma*Tc**4
        
        Eq4 = Fa - Fd + epsilon_3*sigma*T4**4 - 2*epsilon_3*sigma*T3**4

        Eq5 = Fo + Fd + epsilon_3*sigma*T3**4 - sigma*T4**4
        
        Eq6_WTG = T2 - T3      # weak temperature gradient assumption

        Eq6_WBG = Tv(T2,mass_ratio,(q2/(1-q2))) - Tv(T3,mass_ratio,(q3/(1-q3)))   # weak buoyancy assumption

        Eq7_WTG = Cp1*T1 + L*q1 - (Cp2*T2+L*q2_s + g*Z_a)  # convective neutrality by temperature
        # convective neutrality by buoyancy (calculate q1,q2 based on T or Tv?)
        Eq7_WBG = Cp1*Tv(T1,mass_ratio,(q1/(1-q1))) + L*q1 - (Cp2*Tv(T2,mass_ratio,(q2/(1-q2)))+L*q2_s + g*Z_a)
        
        Flux1 = (1-epsilon_2)*sigma*T1**4
        Flux2 = epsilon_2*sigma*T2**4
        Flux3 = epsilon_3*sigma*T3**4
        Flux4 = (1-epsilon_3)*sigma*T3**4
        
        if assumption == 'WTG':                         
            Eq6 = Eq6_WTG
            Eq7 = Eq7_WTG                       
        elif assumption == 'WBG':
            Eq6  = Eq6_WBG
            Eq7 = Eq7_WBG
                
        if index==0:
            return (Eq2, Eq3, Eq4, Eq5, Eq6, Eq7)
        if index==1:
            return(q1, q2, q3, epsilon_2, epsilon_3, Flux1, Flux2, Flux3, Flux4)
    
    # Solve the equations using fsolve function
    index=0
    (T1, T2, T3, T4, Fa, Fc), info, ier, msg = fsolve(equations, p_guess, full_output=True)
    if ier==1:  # a solution is found
        print ('We have a solution w/ conv. mixing: ', (T1, T2, T3, T4, Fa, Fc))
        Es = T1**4 + T4**4
        Ea = T2**4 + T3**4 
        print('The average atmosphere-surface energy ratio is:', (Ea/Es))
        if Fc <= 0:
            print('Yet the solution is incorrect, solve without convection')
        else:
            print('\n')
    
    # Extract the additional terms
    index=1
    #print('Solved.')
    q1,q2,q3,epsilon_2,epsilon_3, Flux1, Flux2, Flux3, Flux4 = equations((T1, T2, T3, T4, Fa, Fc))
    
    fc = cld_frc(Fc)
    alpha_p = alpha(cld_frc(Fc))
    a_2 = a(T2)
    FluxC = - (fc * (Flux1 + Flux2) - fc * sigma*Tc**4)    # cloud radiative effect
    # dayside outgoing longwave radiation
    OLR_day = fc*sigma*Tc**4 +(1 - epsilon_2)*(1-fc)*sigma*T1**4 + (1-fc)*epsilon_2*sigma*T2**4
    # nightside outgoing longwave radiation
    OLR_night = (1 - epsilon_3)*sigma*T4**4 + epsilon_3*sigma*T3**4
    
    # Ourput everything    
    if output == 'default': # Clean mode, only output prescribed variables
        return (T1, T2, T3, T4, Fa, Fc, fc, q1, q2, q3, alpha_p, epsilon_2, epsilon_3, a_2, OLR_day, OLR_night, Flux1, Flux2, Flux3, Flux4, FluxC)
    elif output == 'short':
        return (T1, T2, T3, T4, Fa, Fc, fc, alpha_p, epsilon_2, epsilon_3, a_2)
    elif output == 'debug':            # Debug mode, output the messages from fsolve function
        return (T1, T2, T3, T4, Fa, Fc, fc, q1, q2, q3, alpha_p, epsilon_2, epsilon_3, a_2, OLR_day, OLR_night, Flux1, Flux2, Flux3, Flux4, FluxC, info,ier,msg)